In [14]:
import pandas as pd
df = pd.read_csv("./Data/TOTAL.csv")
# 새로운 열 AN을 생성하고, A에 있는 열의 값이 K1의 열의 값과 같다면 0, K2와 같다면 1, K3와 같다면 2, K4와 같다면 3로 설정
# AN 열 생성 및 값 설정
def set_an_value(row):
    if row['A'] == row['K1']:
        return 0
    elif row['A'] == row['K2']:
        return 1
    elif row['A'] == row['K3']:
        return 2
    elif row['A'] == row['K4']:
        return 3
    else:
        return None  # 예외 처리 필요 시 설정

df['AN'] = df.apply(set_an_value, axis=1)
# 중복된 행 제거
df = df.drop_duplicates()



df.isna().sum()
# nan값 없애기
df = df.dropna()
df.isna().sum()


Q     0
K1    0
K2    0
K3    0
K4    0
A     0
AN    0
dtype: int64

In [15]:
# 각 튜플 안에는 (A, AN, Q+k1+k2+k3+k4)로 구성된 리스트 생성
result_list = []
for index, row in df.iterrows():
    A = row['A']
    AN = row['AN']
    Q = row['Q']
    combined_Q_choices = f"{Q} {row['K1']} {row['K2']} {row['K3']} {row['K4']}"
    result_list.append((A, AN, combined_Q_choices))


In [16]:
import torch
from torchtext.datasets import CoLA
from transformers import AutoTokenizer
from torch.utils.data import DataLoader


def collator(batch, tokenizer, device):
    source, labels, texts = zip(*batch)
    tokenized = tokenizer(
        texts,
        padding="longest",
        truncation=True,
        return_tensors="pt"
    )
    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    return input_ids, attention_mask, labels

# result.txt를 읽은 내용을 train_data에 넣기
train_data = result_list

valid_data = train_data
test_data = train_data

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

epochs = 100
batch_size = 64
device = "cuda" if torch.cuda.is_available() else "cpu"

train_dataloader = DataLoader(
    train_data,
    batch_size=batch_size,
    collate_fn=lambda x: collator(x, tokenizer, device),
    shuffle=True,
)
valid_dataloader = DataLoader(
    valid_data, batch_size=batch_size, collate_fn=lambda x: collator(x, tokenizer, device)
)
test_dataloader = DataLoader(
    test_data, batch_size=batch_size, collate_fn=lambda x: collator(x, tokenizer, device)
)

print("Train Dataset Length :", len(train_data))
print("Valid Dataset Length :", len(valid_data))
print("Test Dataset Length :", len(test_data))


Train Dataset Length : 22939
Valid Dataset Length : 22939
Test Dataset Length : 22939


In [17]:
train_data


[('album', 3.0, '写真帳」を英語でいうと何 ? loose leaf account book nOte album'),
 ('あらまき鮭', 3.0, '鮭の内臓をとって、塩漬けにしたものを何という ? さらまき鮭 はらまき鮭 からまき鮮 あらまき鮭'),
 ('ハーフマラフソン',
  1.0,
  '次のうち、約21kmの距離で行われるマラソン競技はどれ ?  _ フルマフソン ハーフマラフソン ウルトラマラソン クォーターマラソン'),
 ('うちわ', 1.0, '秘の暑い時に、風を起こすために用いるものはどれ ?2 うちきさ うちわ うちこ うちみ'),
 ('下関市', 1.0, '関門海峡を挟んで九州と面している山口県の都市はどこ ? 上関市 下関市 右関市"6 左関市'),
 ('モモレンジャー',
  1.0,
  '「秘密戦際ゴレンジャー』の5人のメンバーのうち、ただ1人の女性は何レンジャー ? キレンジャー モモレンジャー アオレンジー ミドレンジャー'),
 ('詐欺師', 0.0, '漫画『「クロサギ』の主人公・黒崎の職業は何2 詐欺師 闇金の福長 ハッカー 刑事      y : 1303'),
 ('トースト', 0.0, '食パンを、焦げ目がつく程度に焼くことを何と言う? トースト イースト バースト yiit81 コースト'),
 ('山田太郎', 1.0, '野球漫画『ドカベン』の主人公の名前は何 ?            - sp礎一郎 山田太郎 佐藤拓郎 es 高橋義朗'),
 ('九州地方', 1.0, '次のうち、江戸時代に、島原の乱が起こった地方はどこ ? 四国地方 九州地方 近畿地方 東北地方'),
 ('有]買天',
  1.0,
  '大星日に起きた奇跡を描いた三谷幸喜監督の映画『THE 〇〇〇ホテル』。〇〇〇に入る言葉は何 ? 土壇場 有]買天 無表情 \x0c'),
 ('無量大数', 3.0, '次のうち、「10の68乗」を意味する数の単位はどれ ?  - 無茶大数 無理大数 無駄大数 無量大数'),
 ('ブクブク', 0.0, '次のうち、厚着をしたり、太ったりしている様子を意味する言葉はどれ? ブクブク パクパク ビクビク ガクガク'),
 ('LOST',
  0.0,
  '飛行

In [18]:
from torch import optim
from transformers import GPT2ForSequenceClassification


model = GPT2ForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="gpt2",
    num_labels=5
).to(device)
model.config.pad_token_id = model.config.eos_token_id
optimizer = optim.Adam(model.parameters(), lr=5e-5)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
import numpy as np
import torch
from torch import nn
from tqdm import tqdm

def calc_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0

    for input_ids, attention_mask, labels in tqdm(dataloader, desc="Training"):
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(dataloader)
    return train_loss

def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        criterion = nn.CrossEntropyLoss()
        val_loss, val_accuracy = 0.0, 0.0
        
        for input_ids, attention_mask, labels in tqdm(dataloader, desc="Evaluation"):
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits

            loss = criterion(logits, labels)
            logits = logits.detach().cpu().numpy()
            label_ids = labels.to("cpu").numpy()
            accuracy = calc_accuracy(logits, label_ids)
            
            val_loss += loss
            val_accuracy += accuracy
    
    val_loss = val_loss/len(dataloader)
    val_accuracy = val_accuracy/len(dataloader)
    return val_loss, val_accuracy


worst_accuracy = 0
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss, val_accuracy = evaluation(model, valid_dataloader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Val Accuracy {val_accuracy:.4f}")

    if val_accuracy > worst_accuracy:
        best_loss = val_loss
        torch.save(model.state_dict(), "../GPT2ForSequenceClassification.pt")
        print("Saved the model weights")


Training:   0%|          | 0/1434 [00:00<?, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_24428\2295112470.py:17: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  labels = torch.tensor(labels, dtype=torch.long).to(device)
Training:  12%|█▏        | 170/1434 [00:19<02:28,  8.51it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 10.37 GiB is allocated by PyTorch, and 806.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# 예측하기

In [6]:
from transformers import GPT2ForSequenceClassification
import torch

# device 지정
device = "cuda" if torch.cuda.is_available() else "cpu"


model = GPT2ForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="gpt2",
    num_labels=5
).to(device)
model.config.pad_token_id = model.config.eos_token_id
model.load_state_dict(torch.load("./Model/GPT2.pt", map_location=torch.device('cpu')))


#test_loss, test_accuracy = evaluation(model, test_dataloader)
#print(f"Test Loss : {test_loss:.4f}")
#print(f"Test Accuracy : {test_accuracy:.4f}")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [8]:
from transformers import AutoTokenizer
# 모델 예측하기
def predict(model, tokenizer, device, sentence):
    model.eval()
    inputs = tokenizer(sentence, return_tensors="pt", padding=True).to(device)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    return torch.argmax(logits).item()

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
sentence = "野球で「完全試合」といった時、打たれたヒットの数はいくつ?,3本,1本,2本,0本"
predict(model, tokenizer, device, sentence)


3